In [10]:
"""
If you want to use new map results, then go to tournament_reduce.py, and replace the extract() method with the following code. 
This will save the map results for use in this notebook. Once those are saved, switch the original code back. 
"""
"""

async def extract(
    self, results: list[LookupOutput], district: District, term: str
):
    # We first map extraction across all pages.
    results = []
    empty_results = []
    map_pickle = []
    async for r in super().extract(pages, district, term):
        map_pickle.append(r)
        if (r.output is not None) and r.output.extracted_text:
            results.append(r)
        else:
            empty_results.append(r)
            
    # Load existing data if the file already exists
    try:
        with open("./ryurtyn/map_results_11_14_0.dat", "rb") as f:
            existing_data = pickle.load(f)
    except FileNotFoundError:
        existing_data = {}  # If the file doesn't exist, initialize an empty dictionary

    try:
        with open("./ryurtyn/districts_11_14_0.dat", "rb") as f:
            existing_data_district = pickle.load(f)
    except FileNotFoundError:
        existing_data_district = {} 
    # Assuming district.short_name and results are defined somewhere in your code
    new_data = {district.short_name: map_pickle}
    new_data_district = {district.short_name: district}

    # Merge the existing data with the new data
    existing_data.update(new_data)
    existing_data_district.update(new_data_district)

    # Write the updated data back to the file
    with open("./ryurtyn/map_results_11_14_0.dat", "wb") as f:
        pickle.dump(existing_data, f)

    with open("./ryurtyn/districts_11_14_0.dat", "wb") as f:
        pickle.dump(existing_data_district, f)

    # Ensure that we yield one empty result to handle case when the expected output is None
    if len(empty_results) != 0:
        yield empty_results[0]

"""

'\n\nasync def extract(\n    self, results: list[LookupOutput], district: District, term: str\n):\n    # We first map extraction across all pages.\n    results = []\n    empty_results = []\n    map_pickle = []\n    async for r in super().extract(pages, district, term):\n        map_pickle.append(r)\n        if (r.output is not None) and r.output.extracted_text:\n            results.append(r)\n        else:\n            empty_results.append(r)\n            \n    # Load existing data if the file already exists\n    try:\n        with open("./ryurtyn/map_results_11_14_0.dat", "rb") as f:\n            existing_data = pickle.load(f)\n    except FileNotFoundError:\n        existing_data = {}  # If the file doesn\'t exist, initialize an empty dictionary\n\n    try:\n        with open("./ryurtyn/districts_11_14_0.dat", "rb") as f:\n            existing_data_district = pickle.load(f)\n    except FileNotFoundError:\n        existing_data_district = {} \n    # Assuming district.short_name and res

In [11]:
import pickle
from zoning.term_extraction.extract.tournament_tester import TestTournamentReduceExtractor, tournament_reduce
from zoning.term_extraction.eval_results import clean_string_units
import asyncio
import pandas as pd

ModuleNotFoundError: No module named 'zoning.term_extraction.extract.tournament_reduce_tester'

In [ ]:
# load the map outputs
with open("map_results_11_14_0.dat", "rb") as f:
    map_outputs = pickle.load(f)

# load the dictionary of district objects
with open("districts_11_14_0.dat", "rb") as f:
    districts = pickle.load(f)

In [ ]:
# get districts 
districts_without_ground_truth = ["TD", "IP", "VDO", "IND-1", "GB", "MXN", "MX2", "AZ"]
districts_with_ground_truth = [d for d in districts.keys() if d not in districts_without_ground_truth]
len(districts_without_ground_truth), len(districts_with_ground_truth)

(8, 20)

In [ ]:
# Define an asynchronous function that executes the coroutine object
async def execute_tournament_test(inputs, term, district):
    # Your coroutine object (replace this with your actual coroutine)
    async def my_coroutine():
        return await tournament_reduce(inputs, term, district, 1)
    # Execute the coroutine object using await
    return await my_coroutine()

In [ ]:
"""
When you're ready to run the experiment, replace the extract method of tournament with this code.
This ensure that you don't map extract, and just run tournamnet on your pair of elements
"""

"""
async def extract(
    self, pages: list[LookupOutput], district: District, term: str
):
    results = pages
    for result in await tournament_reduce(results, term, district, self.k):
        yield result
"""

'\nasync def extract(\n    self, pages: list[LookupOutput], district: District, term: str\n):\n    results = pages\n    for result in await tournament_reduce(results, term, district, self.k):\n        yield result\n'

Comparing Results Where GT doesn't exist

In [ ]:
total_length = 0
total_selected_index = 0
total_selected_none = 0


for d in districts_without_ground_truth:
    inputs = map_outputs[d]
    district = districts[d]
    term = "min_lot_size"
    print("District: ", district)

    selected_index = 0
    selected_none = 0
    total_length += len(inputs)
    for a in range(len(inputs)):
        i = inputs[a]
        if i.output and i.output.extracted_text:
            for b in range(a + 1, len(inputs)):
                j = inputs[b]
                if j.output and j.output.extracted_text:
                    res = await execute_tournament_test([i, j], term, district)
                    if res == -1:
                        selected_none += 1
                        total_selected_none += 1
                        
                    elif res == 1 or res == 0:
                        selected_index += 1
                        total_selected_index += 1
                    

    print("Selected None: ", selected_none)
    print("Selected Index: ", selected_index)
    print("Accuracy: ", selected_none / len(inputs))

print("Final Selected None: ", total_selected_none)
print("Final Accuracy: ", total_selected_none / total_length)

District:  full_name='Technology Development' short_name='TD'
Selected None:  0
Selected Index:  3
Accuracy:  0.0
District:  full_name='Industrial Park' short_name='IP'


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:37: UserWarning: Phrase CELL (1, 1): 
Minimum lot area was found more than once in the document.
  warnings.warn(f"Phrase {phrase} was found more than once in the document.")


Selected None:  0
Selected Index:  28
Accuracy:  0.0
District:  full_name='Village Overlay' short_name='VDO'


CancelledError: 

Comparing Results Where GT exists

In [ ]:
gt = pd.read_csv("../data/ground_truth.csv")
gt_first_30 = gt.head(30)

In [ ]:
total_length = 0
total_selected_correct = 0
total_selected_incorrect = 0


for d in districts_with_ground_truth:
    inputs = map_outputs[d]
    district = districts[d]
    print(d)
    true_val = gt_first_30[gt_first_30["district_abb"] == d]["min_lot_size_gt"].values[0]
    try:
        true_val = int(true_val)
    except ValueError:
        print("skipping ", d, "true bal is", true_val)
        continue

    term = "min_lot_size"
    print("District: ", district)

    selected_correct = 0
    selected_incorrect = 0
    num_results = 0
    # total_length += len(inputs)
    # find which one is true answer
    true_index = None
    for a in range(len(inputs)):
        i = inputs[a]
        if i.output and i.output.answer:
            # print(i.output.answer)
            i_answer_clean = clean_string_units(i.output.answer)
            print("GT answer: ", true_val, "Cleaned Answer: ", i_answer_clean, "Raw Answer: ", i.output.answer)
            if i_answer_clean:
                i_answer_clean = i_answer_clean[0]
                if i_answer_clean == true_val:
                    true_index = a
                    continue
    # if you found right answer, that becomes the 0th index you pass in to tournament
    if true_index:
        for a in range(len(inputs)):
            if a != true_index:
                i = inputs[a]
                if i.output and i.output.extracted_text:
                    res = await execute_tournament_test([inputs[true_index], i], term, district)
                    total_length += 1
                    num_results += 1
                    if res == 0:
                        selected_correct += 1
                        total_selected_correct += 1 
                    if res == 1:
                        selected_incorrect += 1
                        total_selected_incorrect += 1
        print("Selected Correct: ", selected_correct)
        print("Selected Incorrect: ", selected_incorrect)
        print("Accuracy: ", selected_correct / num_results)
    else:
        print("true answer not found in answers?? check this ")


print("Final Selected Correct: ", total_selected_correct)
print("Final Accuracy: ", total_selected_correct / total_length)

AL
skipping  AL true bal is nan
AA
District:  full_name='AA Residence' short_name='AA'
GT answer:  30000 Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:  30000 Cleaned Answer:  [20000.0] Raw Answer:  20,000 sq ft
GT answer:  30000 Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:  30000 Cleaned Answer:  [653400.0] Raw Answer:  15 acres
GT answer:  30000 Cleaned Answer:  [12500.0] Raw Answer:  12,500 sq ft
GT answer:  30000 Cleaned Answer:  [7500.0] Raw Answer:  7,500 sq ft
GT answer:  30000 Cleaned Answer:  [187308.0] Raw Answer:  4.3 acres
Selected Correct:  6
Selected Incorrect:  0
Accuracy:  1.0
B-1
District:  full_name='Restricted Business' short_name='B-1'
GT answer:  43560 Cleaned Answer:  [87120.0] Raw Answer:  2 acres
GT answer:  43560 Cleaned Answer:  [43560.0] Raw Answer:  1 acre
GT answer:  43560 Cleaned Answer:  [40000.0] Raw Answer:  40,000 sq ft
GT answer:  43560 Cleaned Answer:  [130000.0] Raw Answer:  130,000 sq ft
GT answer:  43560 Cleaned

/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:23: UserWarning: Phrase The maximum allowable density within an HOD District shall be one residential home per gross acre of the combined lot area. was not in the supplied document: 
NEW PAGE 70
Section 9
SECTION 9A - Housing Opportunity Development (This Section Repealed - August 15, 2014)
SECTION 9B - Housing Opportunity Development District (Effective July 1, 2014)
- 63 -
CELL (1, 1): 
9B.1
CELL (1, 2): 
Intent and Purpose:
This Housing Opportunity Development ("HOD") District is intended to increase in the
Town of Bethany the supply of housing that is within the economic means of moderate
income households. The HOD District is a separate and independent zoning district, and
shall replace, not supplement, the existing zoning district of any property to which it is
applied.
CELL (2, 1): 
9B.2 Permitted Uses:
CELL (2, 2): 
9B.2 Permitted Uses:
CELL (3, 1): 
CELL (3, 2): 
The following uses sha

Selected Correct:  4
Selected Incorrect:  2
Accuracy:  0.6666666666666666
C
District:  full_name='Commercial' short_name='C'
GT answer:  10000 Cleaned Answer:  [130680.0] Raw Answer:  130,680 sq ft
GT answer:  10000 Cleaned Answer:  [130680.0] Raw Answer:  130,680 sq ft
GT answer:  10000 Cleaned Answer:  [130680.0] Raw Answer:  130,680 sq ft
true answer not found in answers?? check this 
I-30
District:  full_name='Industrial-30' short_name='I-30'
GT answer:  30000 Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:  30000 Cleaned Answer:  [80000.0] Raw Answer:  80,000 sq ft
GT answer:  30000 Cleaned Answer:  [80000.0] Raw Answer:  80,000 sq ft
GT answer:  30000 Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:  30000 Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:  30000 Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:  30000 Cleaned Answer:  [1.33, 30.0] Raw Answer:  1.33 times the base min lot area for I-30 District


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:23: UserWarning: Phrase 11.15 (7/91) Industrial-30 District. (Section 11.15 through 11.18 deleted and re-assigned to Section 9.5 effective May 1, 2010.) was not in the supplied document: 
NEW PAGE 49
11.8.10 The application for the campground shall include, but not be limited to, the
following: (2/1/12)
11.9. Elderly Housing. The purpose of this Section is to provide opportunities for the
establishment of housing specifically designed and intended for use by the elderly in the R-2
Residential District as a special exception under conditions that consider the special health,
safety and general welfare of this element of the population. Housing for the elderly shall meet
the following conditions:
11.9.1 The property shall include at least five (5) acres.
11.9.2 The maximum number of units shall be four (4) units per acre.
11.9.3 All units will be served by a public water supply system.
11.9.4 In 

Selected Correct:  2
Selected Incorrect:  7
Accuracy:  0.2222222222222222
R-3
District:  full_name='R-3 Residential' short_name='R-3'
GT answer:  15000 Cleaned Answer:  [87120.0] Raw Answer:  87,120 sq ft
GT answer:  15000 Cleaned Answer:  [21780.0] Raw Answer:  0.50 acres
GT answer:  15000 Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:  15000 Cleaned Answer:  [87120.0] Raw Answer:  87,120 sq ft
GT answer:  15000 Cleaned Answer:  [87120.0] Raw Answer:  87,120 sq ft
true answer not found in answers?? check this 
MF
District:  full_name='Multifamily Residence' short_name='MF'
GT answer:  130680 Cleaned Answer:  [130680.0] Raw Answer:  130,680 sq ft
GT answer:  130680 Cleaned Answer:  [174240.0] Raw Answer:  4 acres
GT answer:  130680 Cleaned Answer:  [10000.0] Raw Answer:  10,000 sq ft
GT answer:  130680 Cleaned Answer:  [4500.0] Raw Answer:  4,500 sq ft
GT answer:  130680 Cleaned Answer:  [] Raw Answer:  No specific value provided
true answer not found in answers?? chec

...

Selected Correct:  6
Selected Incorrect:  0
Accuracy:  1.0
SB-C
District:  full_name='Business C' short_name='SB-C'
GT answer:  4500 Cleaned Answer:  [4500.0] Raw Answer:  4,500 sq ft
GT answer:  4500 Cleaned Answer:  [4000.0] Raw Answer:  4,000 sq ft
GT answer:  4500 Cleaned Answer:  [4500.0] Raw Answer:  4,500 sq ft
GT answer:  4500 Cleaned Answer:  [4500.0] Raw Answer:  4,500 sq ft
Selected Correct:  4
Selected Incorrect:  0
Accuracy:  1.0
R-40
District:  full_name='Residential R-40' short_name='R-40'
GT answer:  40000 Cleaned Answer:  [40000.0] Raw Answer:  40,000 sq ft
GT answer:  40000 Cleaned Answer:  [25000.0] Raw Answer:  25,000 sq ft
GT answer:  40000 Cleaned Answer:  [5000.0] Raw Answer:  5,000 sq ft
GT answer:  40000 Cleaned Answer:  [87120.0, 348480.0, 174240.0, 348480.0] Raw Answer:  2 acres (if maximum density is 8 units per acre); 4 acres (if maximum density is greater than 8 units per acre)
GT answer:  40000 Cleaned Answer:  [21780.0] Raw Answer:  1/2 acre
GT answer:  

/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:37: UserWarning: Phrase CELL (1, 1): 
Minimum Lot Size was found more than once in the document.
  warnings.warn(f"Phrase {phrase} was found more than once in the document.")
/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:23: UserWarning: Phrase the minimum area of any lot on which such animals are kept shall be 1.5 acres for one animal, plus 1.0 additional acre for each additional animal. was not in the supplied document: 
NEW PAGE 61
6. A shed which is less than ten (10) feet in height at the roof ridge and less than eighty (80)
square feet in ground floor area may be located within a required side or rear yard, but no
closer than ten (10) feet to any lot line.
6.4.
Keeping of Equine Animals
1. Equine animals (including horses, donkeys, burros and their young) may be kept in any
zone, provided the minimum area of any lot on which such animals are 

Selected Correct:  6
Selected Incorrect:  1
Accuracy:  0.8571428571428571
VC
District:  full_name='Village Commercial' short_name='VC'
GT answer:  60000 Cleaned Answer:  [60112.799999999996, 60000.0] Raw Answer:  1.38 acres (60,000 sq ft)
GT answer:  60000 Cleaned Answer:  [60000.0] Raw Answer:  60,000 sq ft
GT answer:  60000 Cleaned Answer:  [304920.0] Raw Answer:  7 acres
GT answer:  60000 Cleaned Answer:  [130680.0] Raw Answer:  3 acres
GT answer:  60000 Cleaned Answer:  [87120.0] Raw Answer:  2 acres
GT answer:  60000 Cleaned Answer:  [435600.0] Raw Answer:  10 acres
GT answer:  60000 Cleaned Answer:  [435600.0] Raw Answer:  10 acres
GT answer:  60000 Cleaned Answer:  [60000.0] Raw Answer:  60,000 sq ft


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:37: UserWarning: Phrase CELL (2, 1): 
Lot Area was found more than once in the document.
  warnings.warn(f"Phrase {phrase} was found more than once in the document.")
/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:23: UserWarning: Phrase Minimum requirement of ten (10) acres may be waived provided the parcel is abutting or directly across the street from an existing Village Commercial District and the parcel must have at least sixty-thousand square feet of contiguous area which is buildable was not in the supplied document: 
NEW PAGE 31
G.
Ramps shall be provided to permit easy movement of wheelchairs and all codes
relating to the handicapped shall be satisfied.
H.
All units shall be connected to parking areas, recreation facilities and sidewalks
by paved walkways wide enough to accommodate wheelchairs; they shall be a
minimum of five (5) feet in wi

Selected Correct:  6
Selected Incorrect:  1
Accuracy:  0.8571428571428571
Final Selected None:  68
Final Accuracy:  0.8607594936708861
